- After Training the model We found out how much imbalanced Data Can effect the model, Which led us to implement the data augmentation concept to make all the number of images in all files the same 


## number of images we have before data augmentation

In [ ]:
import os
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [ ]:

def count_images_in_directory(directory_path):
    total_images = 0
    for root, dirs, files in os.walk(directory_path):
        image_files = [file for file in files if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
        total_images += len(image_files)
    return total_images

# Paths to your directories
# Set the directories to train and test and validate 
train_dir = r"C:\Users\khair\Downloads\archive (2)\train"
test_dir = r"C:\Users\khair\Downloads\archive (2)\test"
validation_dir = r"C:\Users\khair\Downloads\archive (2)\validation"

# Count images in each directory
train_count = count_images_in_directory(train_dir)
test_count = count_images_in_directory(test_dir)
validation_count = count_images_in_directory(validation_dir)

print(f"Number of images in train directory: {train_count}")
print(f"Number of images in test directory: {test_count}")
print(f"Number of images in validation directory: {validation_count}")


In [ ]:

def count_images_in_class_folders(train_dir):
    class_counts = {}
    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_name)
        if os.path.isdir(class_path):
            image_files = [file for file in os.listdir(class_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
            class_counts[class_name] = len(image_files)
    return class_counts


# Count images in each class within the train directory
train_counts = count_images_in_class_folders(train_dir)


print("Train Directory Image Counts by Class:")
for class_name, count in train_counts.items():
    print(f"{class_name}: {count}")


In [ ]:
# Count images in each class within the test directory
test_counts = count_images_in_class_folders(test_dir)

print("Test Directory Image Counts by Class:")
for class_name, count in test_counts.items():
    print(f"{class_name}: {count}")

In [ ]:
# Count images in each class within the test directory
valid_counts = count_images_in_class_folders(validation_dir)

print("Test Directory Image Counts by Class:")
for class_name, count in valid_counts.items():
    print(f"{class_name}: {count}")

## Data Augmentation method

In [ ]:

def count_images_in_class_folders(directory):
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            image_files = [file for file in os.listdir(class_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
            class_counts[class_name] = len(image_files)
    return class_counts

def augment_data(source_directory, target_directory, target_count, datagen):
    for class_name in os.listdir(source_directory):
        class_path = os.path.join(source_directory, class_name)
        target_class_path = os.path.join(target_directory, class_name)
        os.makedirs(target_class_path, exist_ok=True)
        if os.path.isdir(class_path):
            image_files = [file for file in os.listdir(class_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
            current_count = len(image_files)
            if current_count < target_count:
                for file_name in image_files:
                    img_path = os.path.join(class_path, file_name)
                    img = load_img(img_path)
                    x = img_to_array(img)
                    x = np.expand_dims(x, axis=0)
                    aug_iter = datagen.flow(x, batch_size=1, save_to_dir=target_class_path, save_prefix='aug_', save_format='png')
                    for _ in range(target_count - current_count):
                        next(aug_iter)


# Count images in each class
train_counts = count_images_in_class_folders(train_dir)
test_counts = count_images_in_class_folders(test_dir)
validation_counts = count_images_in_class_folders(validation_dir)

# Determine target count (maximum count among all classes)
max_count = max(max(train_counts.values()), max(test_counts.values()), max(validation_counts.values()))

# Create ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augment data for train, test, and validation directories
augment_data(train_dir, train_dir, max_count, datagen)
augment_data(test_dir, test_dir, max_count, datagen)
augment_data(validation_dir, validation_dir, max_count, datagen)

print("Data augmentation complete. All classes now have equal numbers of images.")
